# Fine-Tuning BLIP on IIW and Comparing to Baseline Model
This notebook fine-tunes BLIP on ImageInWords (IIW) dataset, to generate hyper-detailed descriptions.

##Install the Datasets Library


In [10]:
!pip install datasets accelerate torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 102.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 111.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [11]:
!pip install transformers==4.40.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 123.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 110.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.0 which is incompatible.


In [12]:
!pip install --upgrade transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 138.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 113.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.0
    Uninstalling transformers-4.40.0:
      Successfully uninstalled transformers-4.40.0


#Load Image In Words(IIW) Dataset

In [1]:
import json

dataset_path = '/content/drive/MyDrive/NLP Final Project/data/data.jsonl'

data = []
with open(dataset_path, 'r') as f:
    for line in f:
        data.append(json.loads(line))


Sample Data from IIW (All String entries)

In [2]:
print(f"Loaded {len(data)} entries!")
print(data[0])  # see the first sample


Loaded 400 entries!
{'IIW': "A close-up outdoor shot shows an Echinops Bannaticus Blue Glow Globe flower in front of other flowers of that ilk in front of a blue sky and an out-of-focus background. The flower's spiky extensions are light purple-blue in color with curled tips of dark brown. The stem is pale tan and appears to be fuzzy. At the bottom-left is a clear focused dark green leaf directed toward the viewer with side leaves going out, one to each side. But the bloom high above those leaves is out-of-focus and dark.", 'IIW-P5B': "A detailed close-up captures a bumble bee perched atop a thistle flower, set against a softly blurred backdrop of a clear blue sky. The bee, adorned in a striped coat of yellow and white, is perched on the flower's light brown stem, which extends from the bottom left to the center of the frame. The thistle's purple filaments, in sharp focus, contrast with the blurred background, creating a bokeh effect that draws the viewer's attention to the foreground 

Convert to Dataset object

In [3]:
from datasets import Dataset

# Convert list of dicts to Hugging Face Dataset
dataset = Dataset.from_list(data)

Add corresponding images

In [4]:
from PIL import Image
import os

# Load images based on 'image/key'
image_folder = '/content/drive/MyDrive/NLP Final Project/data/images_aar'

def load_image(example):
    image_path = os.path.join(image_folder, example["image/key"] + '.jpg')
    if os.path.exists(image_path):
        example["image"] = Image.open(image_path).convert("RGB")
    else:
        example["image"] = None  # optional: you could filter these out if missing
    example["caption"] = example["IIW"]  # human hyper-detailed description
    return example

Match images to captions

In [5]:
dataset = dataset.map(load_image)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

# Preprocess Data

In [6]:
dataset = dataset.remove_columns([col for col in dataset.column_names if col not in ["image", "caption"]])
train_test = dataset.train_test_split(test_size=0.1)
train_dataset = train_test["train"]
val_dataset = train_test["test"]

print("Training samples:", len(train_dataset))
print("Validation samples:", len(val_dataset))

Training samples: 360
Validation samples: 40


Load BLIP Processor and Model

In [7]:
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch

# Load BLIP
model_id = "Salesforce/blip-image-captioning-base"

processor = BlipProcessor.from_pretrained(model_id)
model = BlipForConditionalGeneration.from_pretrained(model_id)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Function to Collate data

In [8]:
def collate_fn(batch):
    images = [example["image"] for example in batch]
    captions = [example["caption"] for example in batch]
    inputs = processor(
        images=images,
        text=captions,
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    inputs['labels'] = inputs.input_ids.clone()
    return inputs


In [9]:
import transformers
print(transformers.__version__)

4.51.3


Training arguements

In [10]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir="./blip_finetuned_iiw",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_steps=10,
    num_train_epochs=5,
    learning_rate=5e-5,
    weight_decay=0.01,
    save_total_limit=1,
    fp16=True,
    report_to="none",
    remove_unused_columns=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=collate_fn
)



Train

In [11]:
trainer.train()

Step,Training Loss
10,9.663600
20,7.127100
30,6.209000
40,5.505100
50,4.825600
60,4.185700
70,3.656900
80,3.297100
90,2.727400
100,2.367100


TrainOutput(global_step=225, training_loss=3.126864585876465, metrics={'train_runtime': 403.9906, 'train_samples_per_second': 4.456, 'train_steps_per_second': 0.557, 'total_flos': 1.0681625740050432e+18, 'train_loss': 3.126864585876465, 'epoch': 5.0})

Save Model

In [12]:
model.save_pretrained("/content/drive/MyDrive/NLP Final Project/models/blip_finetuned_iiw_final")
processor.save_pretrained("/content/drive/MyDrive/NLP Final Project/models/blip_finetuned_iiw_final")


[]

# Using The VIM Pipeline

Install dependencies for video processing

In [13]:
!pip install opencv-python

Select Model (Vanilla BLIP, Fine-tuned BLIP, or Llama 4 SCOUT)

In [14]:
import cv2
import os
from transformers import BlipProcessor, BlipForConditionalGeneration, pipeline, AutoProcessor, AutoModelForVision2Seq
from PIL import Image
import torch
import difflib

def load_model(model_choice):
    if model_choice == "blip-vanilla":
        path = "Salesforce/blip-image-captioning-base"
        processor = BlipProcessor.from_pretrained(path)
        model = BlipForConditionalGeneration.from_pretrained(path)
    elif model_choice == "blip-finetuned":
        path = "/content/drive/MyDrive/NLP Final Project/models/blip_finetuned_iiw_final"
        processor = BlipProcessor.from_pretrained(path)
        model = BlipForConditionalGeneration.from_pretrained(path)
    #elif model_choice == "llama-4":
        #path = "meta-llama/Llama-4-Scout-17B-16E-Instruct"  # Future Work - Comparison to SOTA (RELEASED 4/16/25)
        #processor = AutoProcessor.from_pretrained(path)
        #model = AutoModelForVision2Seq.from_pretrained(path)
    else:
        raise ValueError("Invalid model_choice: choose 'blip-finetuned' or 'blip-vanilla'")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    return processor, model, device

Helper Functions

In [15]:
# Function to Extract Frames from Video
def extract_frames(video_path, output_folder, fps=1):
    print("VIW Activated! ")
    print(f"Analyzing...")

    os.makedirs(output_folder, exist_ok=True)
    video = cv2.VideoCapture(video_path)
    frame_rate = video.get(cv2.CAP_PROP_FPS)
    frame_interval = int(frame_rate // fps)

    print("Extracting Frames...")

    count = 0
    saved = 0

    while True:
        ret, frame = video.read()
        if not ret:
            break
        if count % frame_interval == 0:
            frame_filename = os.path.join(output_folder, f"frame_{saved:04d}.jpg")
            cv2.imwrite(frame_filename, frame)
            saved += 1
        count += 1

    video.release()
    print(f"Extracted {saved} frames to {output_folder}")

# Function to Clean the Captions
def clean_caption(raw_caption):
    # Keep everything BEFORE the first </s>
    caption = raw_caption.split("</s>")[0]
    caption = caption.replace("<s>", "").strip()

    # Make sure it ends cleanly
    if not caption.endswith('.'):
        caption += '.'
    caption = caption[0].upper() + caption[1:]

    return caption



# Function to Caption Each Frame
def caption_frames(frame_folder, processor, model, device):
    print("Captioning Frames...")
    captions = []
    frames = sorted(os.listdir(frame_folder))

    for idx, frame_name in enumerate(frames):
        img_path = os.path.join(frame_folder, frame_name)
        raw_image = Image.open(img_path).convert('RGB')

        inputs = processor(images=raw_image, return_tensors="pt").to(device)
        out = model.generate(**inputs, max_length=35, num_beams=3)

        raw_caption = processor.decode(out[0], skip_special_tokens=False)  # DON'T skip tokens yet
        cleaned_caption = clean_caption(raw_caption)  #  run the smart cleaner

        captions.append(cleaned_caption)
        print(f" Frame {idx+1}: {cleaned_caption}")

    return captions



# Funtion to merge frame captions that are at least a percentage different
def merge_captions(frame_captions_unique, similarity_threshold=0.05): # <<< Final Filtering Threshold (Story Smoothing)
    print("\nMerging and Cleaning Captions into a Final Story...")

    unique_texts = []
    last_caption = ""

    for frame_name, caption in frame_captions_unique:
        # Basic cleaning (First Cap and Period at the end)
        caption = caption.strip()
        if not caption.endswith('.'):
            caption += '.'
        caption = caption[0].upper() + caption[1:]

        # Compare similarity to last accepted caption
        similarity = difflib.SequenceMatcher(None, caption.lower(), last_caption.lower()).ratio()

        if similarity < similarity_threshold:
            unique_texts.append(caption)
            last_caption = caption
        else:
            print(f"Skipped Similar Caption: {caption}")

    print('Unique Captions Used: ' + str(len(unique_texts)))
    print(unique_texts)
    combined_text = " ".join(unique_texts)

    print("\nFinal Smoothed Video Caption:")
    print(combined_text)

    return combined_text

# Helper funtion to Run the VIW pipeline
def run_viw_pipeline(video_path, model_choice, fps=.5):
    frames_folder = "/content/frames_output/"

    # Step 1: Load model
    processor, model, device = load_model(model_choice)

    # Step 2: Extract frames
    extract_frames(video_path, frames_folder, fps=fps)

    # Step 3: Caption frames
    frame_captions_full = []   # All generated captions
    frames = sorted(os.listdir(frames_folder))

    print("\nGenerating Captions for Each Frame:")

    for idx, frame_name in enumerate(frames):
        img_path = os.path.join(frames_folder, frame_name)
        raw_image = Image.open(img_path).convert('RGB')

        inputs = processor(images=raw_image, return_tensors="pt").to(device)
        out = model.generate(**inputs, max_length=50)
        caption = processor.decode(out[0], skip_special_tokens=True)

        frame_captions_full.append((frame_name, caption))
        print(f"Frame {idx+1}: {caption}")

    # Step 4: Create Unique Captions based on similarity
    print("\nFiltering Unique Captions Based on Similarity...")
    frame_captions_unique = []
    last_caption = ""

    for frame_name, caption in frame_captions_full:
        caption = caption.strip()
        similarity = difflib.SequenceMatcher(None, caption.lower(), last_caption.lower()).ratio()

        if similarity < 0.25:  # <<< Apply similarity threshold here! ( Early filtering)
            frame_captions_unique.append((frame_name, caption))
            last_caption = caption
        else:
            print(f" Skipped Similar Caption: {caption}")

    # Step 5: Show Unique Captions
    print("\nUnique Captions After Filtering:")
    for idx, (frame_name, caption) in enumerate(frame_captions_unique):
        print(f"Unique {idx+1}: {caption}")

    # Step 6: Merge into Final Story
    combined_text = merge_captions(frame_captions_unique)

    return frame_captions_full, frame_captions_unique, combined_text


# Run VideoInWords (VIW)

In [21]:
# Define available models
model_choices = {
    "Vanilla BLIP": "blip-vanilla",
    "Fine-Tuned BLIP": "blip-finetuned"
    #"LLaMA 4": "llama-4"
}

# Define available videos
video_choices = {
    "A man and his dog walk through the snow in front of a cabin": "/content/drive/MyDrive/NLP Final Project/vids/002.mp4",
    "Close-up View of Chef Preparing Food": "/content/drive/MyDrive/NLP Final Project/vids/003.mp4",
    "Child Playing on a Tranquil Beach": "/content/drive/MyDrive/NLP Final Project/vids/004.mp4",
    "Busy Nightlife Street in Rain with Umbrellas": "/content/drive/MyDrive/NLP Final Project/vids/005.mp4",
    "Tourists at Teotihuacan's Pyramid of the Sun": "/content/drive/MyDrive/NLP Final Project/vids/006.mp4"

}

# 1. Select Model
print("\nAvailable Models:")
for idx, model_name in enumerate(model_choices.keys()):
    print(f"{idx+1}. {model_name}")

model_idx = int(input("\nEnter the number of the model you want to use: ")) - 1
selected_model_key = list(model_choices.keys())[model_idx]
selected_model = model_choices[selected_model_key]

# 2. Select Video
print("\nAvailable Videos:")
for idx, video_name in enumerate(video_choices.keys()):
    print(f"{idx+1}. {video_name}")

video_idx = int(input("\nEnter the number of the video you want to analyze: ")) - 1
selected_video_key = list(video_choices.keys())[video_idx]
selected_video = video_choices[selected_video_key]

# 3. Run Pipeline
print(f"\nRunning VIW with {selected_model_key} on {selected_video_key}...")
full_captions, unique_captions, final_story = run_viw_pipeline(
    video_path=selected_video,
    model_choice=selected_model,
    fps=1
)




Available Models:
1. Vanilla BLIP
2. Fine-Tuned BLIP

Enter the number of the model you want to use: 2

Available Videos:
1. A man and his dog walk through the snow in front of a cabin
2. Close-up View of Chef Preparing Food
3. Child Playing on a Tranquil Beach
4. Busy Nightlife Street in Rain with Umbrellas
5. Tourists at Teotihuacan's Pyramid of the Sun

Enter the number of the video you want to analyze: 1

Running VIW with Fine-Tuned BLIP on A man and his dog walk through the snow in front of a cabin...
VIW Activated! 
Analyzing...
Extracting Frames...
Extracted 22 frames to /content/frames_output/

Generating Captions for Each Frame:
Frame 1: a long shot shows a family in front of a red cabin in the snow with a dog in front of it, a brown dog, and a white cabin in the background, there is a dark blue door, and a white window on the left
Frame 2: a medium outdoor shot shows a group of people and a dog in front of a red cabin in the snow with a dog in front of them, surrounded by ta